여러개의 파일을 Q에다가 쌓은 다음에, 연결한 뒤에, 데이터 양식에 맞게 decode하고 다시 쌓은 뒤에,
학습을 시킬 때 꺼내쓰는 것.

In [1]:
import tensorflow as tf
tf.set_random_seed(777)

1단계

In [2]:
filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename_queue')
#현재 예제에서는 하나의 데이터만 가져온다.

W0814 23:58:11.180225  2284 deprecation.py:323] From <ipython-input-2-9c8b33ae8442>:2: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0814 23:58:11.210975  2284 deprecation.py:323] From C:\Users\13Z970-G.AR30K\Anaconda3\lib\site-packages\tensorflow\python\training\input.py:278: input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0814 23:58:1

2단계

In [3]:
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

W0814 23:58:11.249665  2284 deprecation.py:323] From <ipython-input-3-7c583912b44d>:1: TextLineReader.__init__ (from tensorflow.python.ops.io_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TextLineDataset`.


In [4]:
# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

In [5]:
# collect batches of csv in
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

W0814 23:58:11.312160  2284 deprecation.py:323] From <ipython-input-5-4f47af99acc5>:3: batch (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).


In [6]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [7]:
# Hypothesis
hypothesis = tf.matmul(X, W) + b

In [8]:
# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [9]:
# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [10]:
# Launch the graph in a session.
sess = tf.Session()

In [11]:
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

In [12]:
# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

W0814 23:58:11.659361  2284 deprecation.py:323] From <ipython-input-12-c3fb6c52bbcd>:3: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


In [13]:
for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

0 Cost:  7070.9795 
Prediction:
 [[235.22784]
 [282.40146]
 [278.39618]
 [303.91577]
 [214.62396]
 [159.15425]
 [228.5351 ]
 [170.92644]
 [264.65048]
 [246.59116]]
10 Cost:  5.577785 
Prediction:
 [[154.65848]
 [185.555  ]
 [182.97673]
 [199.99612]
 [140.76091]
 [104.69594]
 [150.7493 ]
 [113.26346]
 [174.13757]
 [162.75192]]
20 Cost:  4.62937 
Prediction:
 [[154.11101 ]
 [184.90294 ]
 [182.33101 ]
 [199.2954  ]
 [140.26302 ]
 [104.333435]
 [150.22685 ]
 [112.87885 ]
 [173.53265 ]
 [162.19771 ]]
30 Cost:  4.618964 
Prediction:
 [[154.10362]
 [184.90007]
 [182.32486]
 [199.29121]
 [140.26024]
 [104.33596]
 [150.22578]
 [112.88076]
 [173.53444]
 [162.20546]]
40 Cost:  4.612851 
Prediction:
 [[154.09985]
 [184.90154]
 [182.32303]
 [199.29173]
 [140.26077]
 [104.34094]
 [150.22823]
 [112.88527]
 [173.54027]
 [162.21696]]
50 Cost:  4.606832 
Prediction:
 [[154.09615]
 [184.90303]
 [182.32123]
 [199.29228]
 [140.26134]
 [104.3459 ]
 [150.23068]
 [112.88977]
 [173.54611]
 [162.22844]]
60 Cost

490 Cost:  4.4063277 
Prediction:
 [[153.9499 ]
 [184.96107]
 [182.24977]
 [199.31444]
 [140.2811 ]
 [104.54182]
 [150.33015]
 [113.07256]
 [173.77664]
 [162.6837 ]]
500 Cost:  4.4030247 
Prediction:
 [[153.94693]
 [184.96225]
 [182.24832]
 [199.31491]
 [140.28145]
 [104.54581]
 [150.33224]
 [113.07639]
 [173.78133]
 [162.69301]]
510 Cost:  4.399764 
Prediction:
 [[153.94398]
 [184.96341]
 [182.24687]
 [199.31538]
 [140.2818 ]
 [104.54976]
 [150.33432]
 [113.08019]
 [173.78598]
 [162.70227]]
520 Cost:  4.396531 
Prediction:
 [[153.94106]
 [184.96454]
 [182.24542]
 [199.31584]
 [140.28215]
 [104.55371]
 [150.33638]
 [113.084  ]
 [173.79063]
 [162.7115 ]]
530 Cost:  4.3933616 
Prediction:
 [[153.93813 ]
 [184.96567 ]
 [182.24399 ]
 [199.3163  ]
 [140.28247 ]
 [104.557625]
 [150.33844 ]
 [113.087776]
 [173.79524 ]
 [162.72066 ]]
540 Cost:  4.3902187 
Prediction:
 [[153.93523]
 [184.96678]
 [182.24254]
 [199.31676]
 [140.28282]
 [104.56153]
 [150.3405 ]
 [113.09155]
 [173.79984]
 [162.7298

1000 Cost:  4.286961 
Prediction:
 [[153.81602]
 [185.01187]
 [182.18306]
 [199.33612]
 [140.29445]
 [104.72166]
 [150.42708]
 [113.25099]
 [173.98833]
 [163.10704]]
1010 Cost:  4.2854486 
Prediction:
 [[153.81372 ]
 [185.0127  ]
 [182.18188 ]
 [199.33649 ]
 [140.29462 ]
 [104.724754]
 [150.42879 ]
 [113.25418 ]
 [173.99194 ]
 [163.11436 ]]
1020 Cost:  4.283993 
Prediction:
 [[153.81143]
 [185.01353]
 [182.18074]
 [199.33688]
 [140.29478]
 [104.72783]
 [150.4305 ]
 [113.25734]
 [173.99556]
 [163.12166]]
1030 Cost:  4.282561 
Prediction:
 [[153.80917 ]
 [185.01437 ]
 [182.1796  ]
 [199.33728 ]
 [140.29495 ]
 [104.73089 ]
 [150.43222 ]
 [113.260506]
 [173.99918 ]
 [163.12894 ]]
1040 Cost:  4.281112 
Prediction:
 [[153.8069 ]
 [185.01521]
 [182.17844]
 [199.33763]
 [140.29512]
 [104.73394]
 [150.43391]
 [113.26366]
 [174.00276]
 [163.13615]]
1050 Cost:  4.279727 
Prediction:
 [[153.80466]
 [185.01604]
 [182.1773 ]
 [199.33803]
 [140.2953 ]
 [104.73696]
 [150.43564]
 [113.26679]
 [174.0063

1490 Cost:  4.239609 
Prediction:
 [[153.71605 ]
 [185.0476  ]
 [182.132   ]
 [199.35353 ]
 [140.30006 ]
 [104.856384]
 [150.50471 ]
 [113.394356]
 [174.14694 ]
 [163.4291  ]]
1500 Cost:  4.239093 
Prediction:
 [[153.71423 ]
 [185.04822 ]
 [182.13107 ]
 [199.35385 ]
 [140.30011 ]
 [104.8588  ]
 [150.50616 ]
 [113.397026]
 [174.14978 ]
 [163.43494 ]]
1510 Cost:  4.2385893 
Prediction:
 [[153.71245]
 [185.04883]
 [182.13014]
 [199.35417]
 [140.30016]
 [104.86121]
 [150.50758]
 [113.3997 ]
 [174.15262]
 [163.44073]]
1520 Cost:  4.2381163 
Prediction:
 [[153.7107 ]
 [185.04944]
 [182.12921]
 [199.35449]
 [140.3002 ]
 [104.86361]
 [150.50903]
 [113.40235]
 [174.15544]
 [163.4465 ]]
1530 Cost:  4.237663 
Prediction:
 [[153.70892]
 [185.05006]
 [182.12833]
 [199.35483]
 [140.30026]
 [104.86599]
 [150.51047]
 [113.40499]
 [174.15826]
 [163.45229]]
1540 Cost:  4.2372136 
Prediction:
 [[153.70717]
 [185.05066]
 [182.12741]
 [199.35515]
 [140.30031]
 [104.86836]
 [150.5119 ]
 [113.40763]
 [174.16

1970 Cost:  4.2289696 
Prediction:
 [[153.63954 ]
 [185.07306 ]
 [182.09189 ]
 [199.3679  ]
 [140.30057 ]
 [104.95979 ]
 [150.56866 ]
 [113.512726]
 [174.26875 ]
 [163.68057 ]]
1980 Cost:  4.22898 
Prediction:
 [[153.63814 ]
 [185.0735  ]
 [182.09114 ]
 [199.36818 ]
 [140.30055 ]
 [104.961685]
 [150.56989 ]
 [113.514984]
 [174.271   ]
 [163.68524 ]]
1990 Cost:  4.229026 
Prediction:
 [[153.63675]
 [185.07394]
 [182.0904 ]
 [199.36847]
 [140.30052]
 [104.96358]
 [150.5711 ]
 [113.51724]
 [174.27321]
 [163.68988]]
2000 Cost:  4.229059 
Prediction:
 [[153.63536 ]
 [185.07439 ]
 [182.08965 ]
 [199.36873 ]
 [140.30048 ]
 [104.96546 ]
 [150.57231 ]
 [113.519485]
 [174.27542 ]
 [163.69449 ]]


In [14]:
coord.request_stop()
coord.join(threads)

In [15]:
# Ask my score
print("Your score will be ",
      sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

Your score will be  [[185.3353]]
Other scores will be  [[178.36246]
 [177.03687]]
